# Identifying Arbitrage Opportunities

This notebook is for testing the individual functions within lambda_script.py, outside of a lambda.

This script identifies arbitrage opportunities across a set of cryptocurrency exchanges by returning the current value of any given cryptocurrency on any given exchange, and identifying the exchanges with the highest and lowest values for said cryptocurrency.

## Setup

In [1]:
# Imports
import json
import operator
import requests
from datetime import datetime

In [13]:
# Declare variables
currency1 = 'BTC'
currency2 = ['USD', 'USDT']

exchanges = [
    "Bitfinex",
    "Binance",
    "Poloniex",
    "Coinbase",
    "BitTrex" 
]

api_key = 'c32565db2e025bfaba2a25ccc2068d5aa44ddaefd51277bfd833e5649606defc'

## Define Functions

In [14]:
def get_coin_price(currency1, currency2, exchange, api_key):
    url = 'https://min-api.cryptocompare.com/data/price?fsym={}&tsyms={}&api_key={}&e={}'.format(
        currency1, 
        currency2, 
        api_key, 
        exchange
    )
    
    headers = {
        'Content-Type': "application/json"
    }
    response = requests.request("GET", url, headers=headers).json()
    
    return response


In [27]:
def find_arbitarage_opportunities(currency1, currency2, exchanges, api_key):
    
    usd_exchange = ['Coinbase', 'BitTrex', 'Bitfinex']
    usdt_exchange = ['Binance', 'Poloniex']
    exchange_list = []
    value_list = []
    
    # create price list
    for exchange in exchanges:
        if exchange in usd_exchange:
            coin_price = get_coin_price(currency1, currency2[0], exchange, api_key)['USD']
        else:
            coin_price = get_coin_price(currency1, currency2[1], exchange, api_key)['USDT']
        value_list.append(coin_price)
        
    # create exchange list
    for i in range(0, len(exchanges)):
        exchange_list.append(exchanges[i])
        
    # create dictionary
    response = dict(zip(exchange_list, value_list))
    
    # min and max
    max_value = max(response.values())
    max_exchange = max(response.items(), key=operator.itemgetter(1))[0]
    min_value = min(response.values())
    min_exchange = min(response.items(), key=operator.itemgetter(1))[0]
    
    # time
    now = datetime.now()
    timestamp = str(now.month) + '-' + str(now.day) + '-' + str(now.year) + ' / ' + str(now.hour) + ':' + str(now.minute) + ':' + str(now.second)
    
    # gains
    dollar_dif = round(max_value - min_value)
    percent_gain = ((max_value - min_value)/min_value)*100
    
    print('percent_gain': percent_gain)
    
    # create output
    output = create_output(
        response, 
        max_value, 
        max_exchange,
        min_value, 
        min_exchange,
        timestamp,
        dollar_dif,
        percent_gain
    )
    
    return output

In [34]:
def create_output(response, max_value, max_exchange, min_value, min_exchange, timestamp, dollar_dif, percent_gain):
    output = {
        'BTC': get_btc_values_and_spreads(response),
        'max_exchange': max_exchange,
        'max_value': max_value,
        'min_exchange': min_exchange,
        'min_value': min_value,
        'timestamp': timestamp,
        'dollar_dif': dollar_dif,
        'percent_gain': percent_gain
    }
    
    return output

In [56]:
def get_btc_values_and_spreads(exchange_and_values):
    
    all_spreads = {}
    
    for key, value in exchange_and_values.items():
        spreads = {}
        temp_key = key
        temp_value = value
        
        for key2, value2 in exchange_and_values.items():
            temp_spread = {
                key2: value-value2
            }
            spreads.update(temp_spread)
        
        temp_dict = {
            temp_key: {
                "value": temp_value,
                "spreads": spreads
            }
        }
        
        all_spreads.update(temp_dict)
        
    return all_spreads

In [57]:
response = find_arbitarage_opportunities(currency1, currency2, exchanges, api_key)

In [59]:
response

{'BTC': {'Binance': {'spreads': {'Binance': 0.0,
    'BitTrex': -23.350000000000364,
    'Bitfinex': -17.550000000000182,
    'Coinbase': -21.300000000000182,
    'Poloniex': -5.579999999999927},
   'value': 7746.65},
  'BitTrex': {'spreads': {'Binance': 23.350000000000364,
    'BitTrex': 0,
    'Bitfinex': 5.800000000000182,
    'Coinbase': 2.050000000000182,
    'Poloniex': 17.770000000000437},
   'value': 7770},
  'Bitfinex': {'spreads': {'Binance': 17.550000000000182,
    'BitTrex': -5.800000000000182,
    'Bitfinex': 0.0,
    'Coinbase': -3.75,
    'Poloniex': 11.970000000000255},
   'value': 7764.2},
  'Coinbase': {'spreads': {'Binance': 21.300000000000182,
    'BitTrex': -2.050000000000182,
    'Bitfinex': 3.75,
    'Coinbase': 0.0,
    'Poloniex': 15.720000000000255},
   'value': 7767.95},
  'Poloniex': {'spreads': {'Binance': 5.579999999999927,
    'BitTrex': -17.770000000000437,
    'Bitfinex': -11.970000000000255,
    'Coinbase': -15.720000000000255,
    'Poloniex': 0.0},
  